In [41]:
from transformers import BertTokenizer, AutoTokenizer

import pandas as pd
import numpy as np

import torch
import torch.nn as nn

#ref: https://mccormickml.com/2019/07/22/BERT-fine-tuning/

In [36]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [38]:
df = pd.read_pickle('../csv/generating_training_df.pickle')
df.columns = ['patch_paths', 'pid', 'cluster_assignment', 'complete_tokens','dtype', 'notes']
df.head()

,patch_paths,pid,cluster_assignment,complete_tokens,dtype,notes
0,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-R55E-1726,4,"[2, pieces, 5x7, mm, broken, apart, good, morp...",train,2 pieces ~9.5x7 mm; 1 broken apart; good morph...
1,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-R55E-1726,4,"[2, pieces, 5x7, mm, broken, apart, good, morp...",train,2 pieces ~9.5x7 mm; 1 broken apart; good morph...
2,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-R55E-1726,4,"[2, pieces, 5x7, mm, broken, apart, good, morp...",train,2 pieces ~9.5x7 mm; 1 broken apart; good morph...
3,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-R55E-1726,4,"[2, pieces, 5x7, mm, broken, apart, good, morp...",train,2 pieces ~9.5x7 mm; 1 broken apart; good morph...
4,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-R55E-1726,4,"[2, pieces, 5x7, mm, broken, apart, good, morp...",train,2 pieces ~9.5x7 mm; 1 broken apart; good morph...


In [39]:
# token_dict = tokenizer.encode_plus()

In [42]:
max_len = 0

# For every sentence...
for sent in df.notes:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  75


In [32]:
token_dict['input_ids'][0]

[101,
 123,
 3423,
 117,
 199,
 126,
 110,
 175,
 2225,
 2599,
 117,
 3687,
 2042,
 2913,
 119,
 2517,
 2817,
 1104,
 1143,
 7301,
 10681,
 25737,
 177,
 7904,
 1643,
 22992,
 117,
 4035,
 1116,
 13284,
 4359,
 102]

In [33]:
print(tokenizer.convert_ids_to_tokens(token_dict['input_ids'][0]))

['[CLS]', '2', 'pieces', ',', '~', '5', '%', 'f', '##as', '##ca', ',', 'del', '##ine', '##ated', '.', 'minute', 'focus', 'of', 'me', '##so', '##the', '##lial', 'h', '##yer', '##p', '##lasia', ',', 'en', '##s', '##qua', '##red', '[SEP]']


In [34]:
def check_cuda():
    if torch.cuda.is_available():       
        device = torch.device("cuda")
        print(f'There are {torch.cuda.device_count()} GPU(s) available.')
        print('Device name:', torch.cuda.get_device_name(0))
        return device
    else:
        print('No GPU available, using the CPU instead.')
        device = torch.device("cpu")
        return device
    
device=check_cuda()

No GPU available, using the CPU instead.


In [ ]:
tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 96,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )